In [1]:
import pickle
import numpy as np
from src.groupedpaneldatamodels import GroupedFixedEffects, GroupedInteractiveFixedEffects
from joblib import Parallel, delayed
import cProfile

# Bonhomme Manresa

In [2]:
def bm_run(run, G, i):
    x, y, f, g = run
    x = np.float32(x)
    y = np.atleast_3d(np.float32(y))
    f = np.float32(f)
    gfe_iterations = 10 if G < 4 else 100
    n_boot = 200 if G < 4 else 100
    m = GroupedFixedEffects(y, x, G, True, hide_progressbar=False).fit(
        max_iter=100, gfe_iterations=gfe_iterations, n_boot=n_boot
    )
    # print(m.params["g"])
    display(m.summary())
    display(m.t_values)
    print(f"Finished run {i}")

    # return m.to_dict()

In [ ]:
for n in (100, 200,):
    for t in (20, 50,):
        for G in (3, 6,):
            for k in (3,):
                print(f"\n\nRunning DGP1 with n={n}, t={t}, G={G}, k={k}")
                runs = pickle.load(open(f"development/generated_data/dgp1_n{n}_t{t}_G{G}_k{k}.pkl", "rb"))
                estimates = []
                estimates = Parallel(n_jobs=-1, prefer="processes")(
                    delayed(bm_run)(run, G, i) for i, run in enumerate(runs[:200])
                )

                with open(f"estimates/dgp1_n{n}_t{t}_G{G}_k{k}.pkl", "wb") as f:
                    pickle.dump(estimates, f)



Running DGP1 with n=100, t=20, G=3, k=3


In [3]:
runs = pickle.load(open("development/generated_data/dgp1_n100_t20_G3_k3.pkl", "rb"))

from statsmodels.api import OLS
OLS(np.float32(runs[0][1].reshape(1, -1)).T, np.float32(runs[0][0].reshape(3, -1)).T).fit().summary2()

# np.float32(runs[0][0].reshape(3, -1)).shape

<class 'statsmodels.iolib.summary2.Summary'>
"""
                        Results: Ordinary least squares
================================================================================
Model:                  OLS              Adj. R-squared (uncentered): -0.001    
Dependent Variable:     y                AIC:                         11321.3493
Date:                   2025-06-21 21:02 BIC:                         11338.1521
No. Observations:       2000             Log-Likelihood:              -5657.7   
Df Model:               3                F-statistic:                 0.2866    
Df Residuals:           1997             Prob (F-statistic):          0.835     
R-squared (uncentered): 0.000            Scale:                       16.799    
--------------------------------------------------------------------------------------
           Coef.        Std.Err.          t          P>|t|         [0.025       0.975]
--------------------------------------------------------------------------------------
x1         0.0828         0.0909        0.9106       0.3626       -0.0955       0.2611
x2        -0.0177         0.0938       -0.1891       0.8500       -0.2018       0.1663
x3         0.0078         0.0930        0.0839       0.9331       -0.1746       0.1902
--------------------------------------------------------------------------------
Omnibus:                  71.244            Durbin-Watson:               1.749  
Prob(Omnibus):            0.000             Jarque-Bera (JB):            149.888
Skew:                     0.220             Prob(JB):                    0.000  
Kurtosis:                 4.267             Condition No.:               1      
================================================================================
Notes:
[1] R² is computed without centering (uncentered) since the
model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly
specified.
"""

In [5]:
# print()
bm_run(runs[0], 3, 0)

Bootstrap GroupedFixedEffects@0x16d49fc50: 100%|██████████| 200/200 [00:04<00:00, 47.11it/s]


<class 'statsmodels.iolib.summary.Summary'>
"""
                        GroupedFixedEffects Summary                        
===========================================================================
               Type    bonhomme_manresa              AIC 210.58679657538903
       Observations                2000              BIC 1123.5338974807485
Exogenous Variables                   3             HQIC  545.8018336590757
             Groups                   3          sigma^2 1.0276830194179982
           Fit Time 2025-06-21 21:02:21             Seed                N/A
       Fit Duration        4.31 seconds    Use Bootstrap               True
  Hetrogeneous Beta                True Confidence Level               0.95
                         beta                         
======================================================
Parameter Coef.  Std.Err.    t    P>|t|  [0.025 0.975]
------------------------------------------------------
  beta0,0 1.0971   0.0468 23.4293 0.0000 1.0053 1.1889
  beta0,1 0.9409   0.0372 25.3276 0.0000 0.8681 1.0137
  beta0,2 0.9447   0.0334 28.2832 0.0000 0.8792 1.0101
                                                      
  beta1,0 1.9583   0.0435 44.9690 0.0000 1.8730 2.0437
  beta1,1 2.0695   0.0365 56.7064 0.0000 1.9980 2.1410
  beta1,2 1.9327   0.0340 56.8313 0.0000 1.8661 1.9994
                                                      
  beta2,0 3.0268   0.0337 89.8995 0.0000 2.9608 3.0928
  beta2,1 3.0232   0.0378 80.0251 0.0000 2.9492 3.0972
  beta2,2 2.9340   0.0423 69.4218 0.0000 2.8512 3.0169
         alpha coef.         
=============================
alpha    0       1       2   
-----------------------------
    0  2.2728 -0.6146 -1.7316
    1  2.4192  0.2658 -2.3536
    2  3.0093   0.346 -0.8102
    3  1.0501 -0.8137 -0.9608
    4  0.7332 -1.6993  0.5016
    5  1.5028 -2.9348  2.1542
    6  0.7222 -0.9955  2.7822
    7 -0.4818   1.019  2.3878
    8  0.4102 -0.3305  2.3775
    9  0.6341  0.2841  1.0747
   10 -1.1197 -0.8407  0.7079
   11  0.4035 -3.1078  1.9754
   12   0.946  -0.715  1.5456
   13  1.4261  0.1184  2.8005
   14   0.218 -1.2254  2.7633
   15 -0.5256 -1.3671  2.8652
   16 -0.7944  -0.928  2.5637
   17 -0.5706 -0.3113  3.5934
   18  0.2726 -0.2224  3.0859
   19 -1.1807  0.9271  2.4673
                                                                    g coef.                                                                    
===============================================================================================================================================
Index                                                                   Value                                                                  
-----------------------------------------------------------------------------------------------------------------------------------------------
    0 [3, 6, 9, 12, 16, 25, 27, 32, 34, 35, 37, 38, 40, 42, 45, 46, 47, 49, 50, 52, 56, 60, 62, 65, 70, 80, 84, 87, 88, 90, 92, 94, 95, 96, 97]
    1             [2, 5, 7, 10, 13, 14, 18, 19, 22, 24, 26, 28, 29, 30, 33, 39, 41, 43, 44, 51, 53, 57, 58, 67, 69, 72, 74, 75, 77, 81, 82, 93]
    2          [0, 1, 4, 8, 11, 15, 17, 20, 21, 23, 31, 36, 48, 54, 55, 59, 61, 63, 64, 66, 68, 71, 73, 76, 78, 79, 83, 85, 86, 89, 91, 98, 99]
-----------------------------------------------------------------------------------------------------------------------------------------------
"""

{'beta': array([[23.4293056 , 25.32757216, 28.28319153],
        [44.96898304, 56.70641059, 56.83132204],
        [89.89952988, 80.02508271, 69.4217897 ]])}

Finished run 0


In [ ]:
def ab_run(run, G, i):
    x, y, f, g, *_ = run
    x = np.float32(x)
    y = np.atleast_3d(np.float32(y))
    f = np.float32(f)
    gife_iterations = 10 if G < 4 else 100
    n_boot = 200 if G < 4 else 100
    m = GroupedInteractiveFixedEffects(y, x, G, True, hide_progressbar=False).fit(max_iter=100, gife_iterations=gife_iterations, n_boot=n_boot)
    print(f"Finished run {i}")
    return m.to_dict()

runs = pickle.load(open("development/generated_data/dgp2_n100_t20_G3_k3.pkl", "rb"))
ab_run(runs[0], 3, 0)

Bootstrap GroupedInteractiveFixedEffects@0x16b035590: 100%|██████████| 200/200 [00:19<00:00, 10.30it/s]

Finished run 0


<class 'statsmodels.iolib.summary.Summary'>
"""
                  GroupedInteractiveFixedEffects Summary                  
==========================================================================
               Type            ando_bai              AIC 4035.976964432682
       Observations                2000              BIC 4965.726772716667
Exogenous Variables                   3             HQIC 4377.361603425885
             Groups                   3          sigma^2 4.900625609210888
           Fit Time 2025-06-21 21:03:17             Seed               N/A
       Fit Duration      188.89 seconds    Use Bootstrap              True
  Hetrogeneous Beta                True Confidence Level              0.95
                         beta                         
======================================================
Parameter Coef.  Std.Err.    t    P>|t|  [0.025 0.975]
------------------------------------------------------
  beta0,0 0.9649   0.1069  9.0296 0.0000 0.7554 1.1743
  beta0,1 0.9800   0.0882 11.1146 0.0000 0.8072 1.1528
  beta0,2 1.0520   0.1049 10.0316 0.0000 0.8465 1.2576
                                                      
  beta1,0 1.9430   0.0610 31.8351 0.0000 1.8234 2.0626
  beta1,1 2.0688   0.0457 45.2540 0.0000 1.9792 2.1584
  beta1,2 1.9295   0.0535 36.0722 0.0000 1.8247 2.0343
                                                      
  beta2,0 3.0243   0.1193 25.3498 0.0000 2.7904 3.2581
  beta2,1 2.9901   0.1571 19.0369 0.0000 2.6823 3.2980
  beta2,2 3.1425   0.1148 27.3692 0.0000 2.9175 3.3676
                                                                      g coef.                                                                      
===================================================================================================================================================
Index                                                                     Value                                                                    
---------------------------------------------------------------------------------------------------------------------------------------------------
    0           [0, 2, 4, 6, 8, 11, 12, 18, 22, 24, 28, 34, 39, 41, 49, 50, 53, 56, 57, 59, 60, 61, 65, 79, 80, 82, 87, 88, 89, 91, 93, 94, 96, 97]
    1 [3, 5, 9, 13, 14, 15, 16, 19, 25, 26, 27, 29, 30, 31, 32, 33, 35, 37, 38, 40, 47, 51, 62, 63, 64, 67, 68, 69, 70, 72, 83, 86, 90, 92, 95, 98]
    2                        [1, 7, 10, 17, 20, 21, 23, 36, 42, 43, 44, 45, 46, 48, 52, 54, 55, 58, 66, 71, 73, 74, 75, 76, 77, 78, 81, 84, 85, 99]
         F coef.         
=========================
F    0       1       2   
-------------------------
0 -0.8154 -0.4995 -1.2662
1   0.739  0.5426  0.4567
2 -0.3956  0.4796   0.983
         Lambda coef.         
==============================
Lambda    0       1       2   
------------------------------
     0 -0.5752  5.0477 -0.6615
     1 -0.1838 -1.9743  1.4635
     2  -0.827  2.7084 -0.5072
     3  3.3847 -0.1284 -0.0888
     4 -0.4679  3.7302 -0.2588
     5  2.3078 -0.3185 -0.2119
     6 -0.1953  4.7518  -1.885
     7 -0.0886 -0.9835   2.317
     8 -0.4232  3.1646 -1.9387
     9  1.6378 -0.2988  0.4574
    10 -0.0166 -0.5093  3.7667
    11 -0.7587  1.8568  0.3775
    12  -0.115  3.4821  0.1363
    13  3.1266 -0.4608 -0.3749
    14  2.8307 -0.0746  0.5771
    15  1.6179 -0.1519 -0.9934
    16  0.9638  0.3445  0.0743
    17 -0.3772  0.1208  3.6313
    18  0.0456  4.6616 -2.0552
    19  1.2357 -0.5284 -0.5669
    20 -0.2361 -0.5563  1.0725
    21  0.3691 -1.0242  1.7807
    22  0.9049  3.7002 -1.1482
    23  0.0389 -1.5223  0.9628
    24 -0.6257  3.6593 -1.1334
    25  1.3159  0.1513  0.2057
    26  1.4381 -0.3855 -0.5571
    27   2.698 -1.3098 -0.1171
    28 -0.0986  3.5512 -2.4489
    29  1.1091 -0.0657  0.1985
    30   2.628  0.3231 -0.2685
    31  2.6039  0.5967 -0.4107
    32  2.3401 -0.4187 -0.2006
    33  1.2987  0.3689  0.0316
    34  0.6119  1.8438 -1.9712
    35   1.032  0.1233 -0

{'name': 'GroupedInteractiveFixedEffects',
 'id': '0x16b035590',
 'fit_datetime': '2025-06-21 21:03:17',
 'fit_duration': 188.89344699999998,
 'model_type': 'ando_bai',
 'params': {'beta': array([[0.9648707 , 0.97999588, 1.05203721],
         [1.94298067, 2.06876911, 1.92950341],
         [3.02425816, 2.99014434, 3.14252544]]),
  'g': {0: [0,
    2,
    4,
    6,
    8,
    11,
    12,
    18,
    22,
    24,
    28,
    34,
    39,
    41,
    49,
    50,
    53,
    56,
    57,
    59,
    60,
    61,
    65,
    79,
    80,
    82,
    87,
    88,
    89,
    91,
    93,
    94,
    96,
    97],
   1: [3,
    5,
    9,
    13,
    14,
    15,
    16,
    19,
    25,
    26,
    27,
    29,
    30,
    31,
    32,
    33,
    35,
    37,
    38,
    40,
    47,
    51,
    62,
    63,
    64,
    67,
    68,
    69,
    70,
    72,
    83,
    86,
    90,
    92,
    95,
    98],
   2: [1,
    7,
    10,
    17,
    20,
    21,
    23,
    36,
    42,
    43,
    44,
    45,
    46,


In [ ]:
def ab_run(run, G, i):
    x, y, f, g, *_ = run
    x = np.float32(x)
    y = np.atleast_3d(np.float32(y))
    f = np.float32(f)
    gife_iterations = 10 if G < 4 else 100
    n_boot = 200 if G < 4 else 100
    m = GroupedInteractiveFixedEffects(y, x, G, True, hide_progressbar=False).fit(max_iter=100, gife_iterations=gife_iterations, n_boot=n_boot)
    print(f"Finished run {i}")

    return m.to_dict()


for n in (100, 200):
    for t in (20, 50):
        for G in (3, 6):
            for k in (3,):
                print(f"\n\nRunning DGP2 (ab) with n={n}, t={t}, G={G}, k={k}")
                runs = pickle.load(open(f"development/generated_data/dgp2_n{n}_t{t}_G{G}_k{k}.pkl", "rb"))
                estimates = []
                estimates = Parallel(n_jobs=-1, prefer="processes")(
                    delayed(ab_run)(run, G, i) for i, run in enumerate(runs[:200])
                )

                with open(f"estimates/dgp2_n{n}_t{t}_G{G}_k{k}.pkl", "wb") as f:
                    pickle.dump(estimates, f)

In [8]:
def sj_run(run, G, i):
    x, y, f, g, *_ = run
    x = np.float32(x)
    y = np.atleast_3d(np.float32(y))
    f = np.float32(f)
    n_boot = 100 if G < 4 else 50
    kappa = 0.005 if G < 4 else 0.00005
    m = GroupedInteractiveFixedEffects(y, x, G, True, "su_ju", R=G, hide_progressbar=True).fit(
        n_boot=n_boot, only_bfgs=True, max_iter=100, kappa=kappa, tol=1e-2
    )
    print(m.IC)
    display(m.summary())
    print(f"Finished run {i}")

    # return m.to_dict()

In [9]:
runs = pickle.load(open(f"development/generated_data/dgp2_n{100}_t{20}_G{3}_k{3}.pkl", "rb"))
sj_run(runs[0], 2, 0)
sj_run(runs[0], 3, 0)
sj_run(runs[0], 4, 0)
sj_run(runs[0], 5, 0)
sj_run(runs[0], 6, 0)

{'sigma^2': np.float64(2.8063673828535514), 'AIC': np.float64(2493.062899623532), 'BIC': np.float64(5327.119544151826), 'HQIC': np.float64(3533.6690883618476)}


<class 'statsmodels.iolib.summary.Summary'>
"""
                   GroupedInteractiveFixedEffects Summary                  
===========================================================================
               Type               su_ju              AIC  2493.062899623532
       Observations                2000              BIC  5327.119544151826
Exogenous Variables                   3             HQIC 3533.6690883618476
             Groups                   2          sigma^2 2.8063673828535514
           Fit Time 2025-06-21 21:03:53             Seed                N/A
       Fit Duration      200.69 seconds    Use Bootstrap               True
  Hetrogeneous Beta                True Confidence Level               0.95
                         beta                         
======================================================
Parameter Coef.  Std.Err.    t    P>|t|  [0.025 0.975]
------------------------------------------------------
  beta0,0 1.1601   0.0972 11.9338 0.0000 0.9696 1.3506
  beta0,1 2.7524   0.1136 24.2379 0.0000 2.5298 2.9749
                                                      
  beta1,0 1.1519   0.1006 11.4461 0.0000 0.9547 1.3492
  beta1,1 2.6872   0.1230 21.8442 0.0000 2.4461 2.9283
                                                      
  beta2,0 1.2172   0.1049 11.6025 0.0000 1.0116 1.4229
  beta2,1 2.7902   0.1292 21.6016 0.0000 2.5370 3.0434
        b coef.         
========================
b    0       1      2   
------------------------
 0  1.101  1.0178 0.8437
 1 2.5001  2.9103 2.9465
 2 0.6333  1.0641 0.9756
 3 2.4692  2.0753  2.203
 4 1.1443  0.5477 0.9806
 5 1.2865  2.9096 1.9973
 6 0.9274  1.3695 1.0083
 7 3.1908  2.9452 3.4059
 8 0.7854  0.8545 0.5709
 9 1.0587  2.8986 1.3753
10 3.1258  3.0869 2.7023
11  1.012  0.9804 0.9085
12 0.7678  1.2203 0.6939
13 2.1254  1.6773 2.0121
14 1.2638 -0.1518 1.7383
15 2.2323  2.1081 1.8868
16 1.9733  2.5058 2.1105
17 2.9287  3.1076 2.9889
18 0.2078  0.8073 1.3424
19 2.4637  2.5149 1.4938
20 2.7921  2.5419 2.9583
21 3.0714  3.0877 3.0916
22 1.3175  0.7622 0.9032
23 3.0721  2.8535 3.1546
24 0.9342  1.1139 1.3821
25 2.3329  1.5469  1.571
26   1.95  1.8084 2.3402
27 2.0709  2.2301 1.7757
28 1.6237  1.0004 1.1488
29 2.0855  2.0098  1.713
30 0.3754  1.5025 1.7777
31 1.6446  1.8899 2.4591
32 1.6075  1.3439 2.1358
33 2.6634  1.9833 1.5571
34 1.4618  1.0046  0.843
35 2.1337  2.0661 1.7677
36 2.8078  2.5312 3.0198
37 1.7374  2.4098 1.8342
38 3.4087  0.5745 3.1634
39 1.1056  0.8266 1.0203
40 1.7969  1.0352 2.4561
41 0.7342  1.0789 0.7427
42 3.0841  2.6314 2.8846
43 2.9227  3.4579 2.5339
44 2.6621  3.1661 2.8755
45 3.0458  2.9719 3.5834
46 3.2786  3.0896 2.6162
47 1.0053 -0.0118 1.6068
48 3.3147  2.8369 2.9503
49  1.038  0.9173  1.138
50 1.1401  1.0637 1.2951
51 2.1288  2.1936 1.4505
52 2.7344  2.4837 3.1089
53  1.018  0.6721  1.462
54 3.1097  3.5111 2.8243
55 3.9827  3.2366 2.7385
56 1.1077  1.1947 1.1993
57 1.0689  1.0178 1.2037
58 2.8785  2.1989 2.9208
59 1.0583   1.007 1.2454
60 1.1216  0.9044 1.2228
61  0.646  0.8317 0.5665
62  1.587  0.8917 1.9903
63 1.7719  1.6345   2.12
64 1.5418  2.7131 2.7171
65  0.907  1.0917 0.3783
66 2.7554  2.7551 2.9995
67 1.1947  2.4008 1.6073
68  2.588   0.603 1.4775
69 3.0441  2.1515 2.5496
70 2.6275  2.0076 2.1486
71 3.1011  3.1124 2.8001
72 2.0132  2.3633 1.7864
73 2.5924  2.3502 3.7887
74 3.1028  3.1975 3.1033
75 3.5912  3.2251 3.1027
76 2.8945  2.8932 3.2297
77 3.1225  3.4316 3.1618
78 2.8668  3.3537 3.0806
79 1.0011  1.2417 1.0483
80 0.8832  1.0247 0.4694
81 2.7569  3.2118 3.0144
82 0.6626  0.9179 0.2811
83 2.3254  1.9263 1.5576
84 3.1313  2.8625 3.1822
85 2.6554  3.0877 3.1375
86 1.8643  2.3649 2.2703
87  1.167  0.8975 0.6835
88 1.1136  1.1395 1.4627
89 0.8592  1.2852  1.301
90 0.8889  1.3726 1.6562
91 1.2827  1.3893 1.1844
92 2.8671  2.5463 2.6456
93 0.6347   1.113 0.7721
94 0.8096  0.8586  0.908
95 2.8927  1.7758 2.4377
96 1.1848  1.1792 1.0966
97 1.5659  1.2533 1.3355
98 0.6931  2.3424 2.1675
99 2.6295  2.9021 3.0989
      la

Finished run 0


KeyboardInterrupt: 

In [ ]:
for n in (100, 200):
    for t in (20, 50):
        for G in (3, 6):
            for k in (3,):
                print(f"\n\nRunning DGP2 (su_ju) with n={n}, t={t}, G={G}, k={k}")
                runs = pickle.load(open(f"development/generated_data/dgp2_n{n}_t{t}_G{G}_k{k}.pkl", "rb"))
                estimates = []
                estimates = Parallel(n_jobs=-1, prefer="processes")(
                    delayed(sj_run)(run, G, i) for i, run in enumerate(runs[:200])
                )

                with open(f"estimates/dgp2_n{n}_t{t}_G{G}_k{k}_su_ju.pkl", "wb") as f:
                    pickle.dump(estimates, f)

In [ ]:
def ssp_run(run, G, i):
    x, y, g = run
    x = np.float32(x)
    y = np.atleast_3d(np.float32(y))
    n_boot = 100 if G < 4 else 50
    kappa = 0.005 if G < 4 else 0.00005
    m = GroupedFixedEffects(y, x, G, False, "su_shi_phillips", hide_progressbar=True).fit(
        n_boot=n_boot, only_bfgs=False, max_iter=100, kappa=kappa, tol=1e-5
    )

    return m.IC

In [ ]:
runs = pickle.load(open(f"development/generated_data/dgp3_n{100}_t{50}_G{3}_k{3}.pkl", "rb"))
print(ssp_run(runs[136], 1, 0))
print(ssp_run(runs[136], 2, 0))
print(ssp_run(runs[136], 3, 0))
print(ssp_run(runs[136], 4, 0))
print(ssp_run(runs[136], 5, 0))
print(ssp_run(runs[136], 6, 0))

{'sigma^2': np.float64(0.9965671526745375), 'AIC': np.float64(369.4860292085887), 'BIC': np.float64(2995.9148853493325), 'HQIC': np.float64(1290.008029452235)}
{'sigma^2': np.float64(0.9972898880409805), 'AIC': np.float64(375.82927511601787), 'BIC': np.float64(3021.8097108310108), 'HQIC': np.float64(1303.2037964532894)}
{'sigma^2': np.float64(0.998089879003516), 'AIC': np.float64(382.5670990182064), 'BIC': np.float64(3048.099114307448), 'HQIC': np.float64(1316.7941414491029)}
{'sigma^2': np.float64(0.9985150295990253), 'AIC': np.float64(387.45708278242535), 'BIC': np.float64(3072.540677645916), 'HQIC': np.float64(1328.5366463069472)}
{'sigma^2': np.float64(0.9991685723776971), 'AIC': np.float64(393.4575427219694), 'BIC': np.float64(3098.092717159708), 'HQIC': np.float64(1341.3896273401165)}
{'sigma^2': np.float64(0.9998228080006262), 'AIC': np.float64(399.4590562177396), 'BIC': np.float64(3123.645810229727), 'HQIC': np.float64(1354.2436619295117)}


In [ ]:
for n in (100, 200):
    for t in (20, 50):
        for G in (3, 6):
            for k in (3,):
                print(f"\n\nRunning DGP3 (su_shi_phillips) with n={n}, t={t}, G={G}, k={k}")
                runs = pickle.load(open(f"development/generated_data/dgp3_n{n}_t{t}_G{G}_k{k}.pkl", "rb"))
                estimates = []
                estimates = Parallel(n_jobs=-1, prefer="processes")(
                    delayed(ssp_run)(run, G, i) for i, run in enumerate(runs[:200])
                )
                with open(f"estimates/dgp3_n{n}_t{t}_G{G}_k{k}.pkl", "wb") as f:
                    pickle.dump(estimates, f)